In [2]:
import pandas as pd
import numpy as np
from random import randint
import math

In [8]:
useful_df = pd.read_csv('C:/Users/J/Desktop/Git_Repositories/desktop-meal-maker/diet_facts_usda.csv', encoding='ISO-8859-2')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
useful_df.head()

food_key                                   food_description  \
0        1                       AARDVARK HABENERO HOT SAUCE,   
1        2          AARON'S BEST, OVEN ROASTED TURKEY BREAST,   
2        3  A&B AMERICAN STYLE, MORE HEAT SMALL BATCH PEPP...   
3        4          A&B AMERICAN STYLE, ORGANIC PEPPER SAUCE,   
4        5          A&B AMERICAN STYLE, PEPPER SAUCE, GARLIC,   

                             brand food_type_grp  \
0  Secret Aardvark Trading Company       grocery   
1     Agri Star Meat & Poultry LLC       grocery   
2         A & B AMERICAN STYLE LLC       grocery   
3         A & B AMERICAN STYLE LLC       grocery   
4                    Namaste Foods       grocery   

                                    ingredients_list serving_size_raw  \
0  TOMATOES (TOMATOES AND FIRE ROASTED TOMATOES, ...          tsp 5g'   
1  TURKEY, WATER, CONTAINS 2% OR LESS DEXTROSE, S...      SLICES 56g'   
2  WHITE VINEGAR, RED CHILI PEPPER, CARROT, HABAN...          ONZ 7g'   
3  ORGANIC WHITE VINEGAR, ORGANIC JALAPENO, ORGAN...          ONZ 7g'   
4  WHITE VINEGAR, RED CHILI PEPPER, CARROT, HABAN...          ONZ 7g'   

  serving_size_val serving_size_unit calories protein_g     ...       \
0              NaN               NaN        0         0     ...        
1              NaN               NaN       60        11     ...        
2              NaN               NaN        0         0     ...        
3              NaN               NaN        0         0     ...        
4              NaN               NaN        0         0     ...        

   saturated_fat_g  carb_g  fiber_g  sugar_g  sodium_mg  cholesterol_mg  \
0              NaN     0.0      NaN      NaN       35.0             NaN   
1              0.0     2.0      NaN      1.0      610.0            30.0   
2              NaN     0.0      NaN      NaN       10.0             NaN   
3              NaN     0.0      NaN      NaN       10.0             NaN   
4              NaN     0.0      NaN      NaN       10.0             NaN   

   Unnamed: 17  Unnamed: 18  Unnamed: 19  Unnamed: 20  
0          NaN          NaN          NaN          NaN  
1          NaN          NaN          NaN          NaN  
2          NaN          NaN          NaN          NaN  
3          NaN          NaN          NaN          NaN  
4          NaN          NaN          NaN          NaN  

[5 rows x 21 columns]

In [9]:
# Randomly assign groups 1 - 5. 
useful_df['rand_group'] = [randint(0,4) for i in range(len(useful_df))]

In [13]:
# Rules: Need 1 from each group.
prot_req = 144
fat_req = 64
carb_req = 132

In [11]:
def add_sorted_item(useful_df, plan_df, delta, proportions):
    delta = np.array([prot_req,fat_req,carb_req])-np.array([sum(plan_df['protein_g']),sum(plan_df['fat_g']),sum(plan_df['carb_g'])])
    proportions = np.array([delta[0]/sum(delta),delta[1]/sum(delta),delta[2]/sum(delta)])
    useful_df['prop_distance'] = np.sqrt((useful_df.prot_prop-proportions[0])**2+(useful_df.fat_prop - proportions[1])**2+(useful_df.carb_prop - proportions[2])**2)# a
    useful_df = useful_df.sort_values(by=['prop_distance'])
# Append closest distance
    plan_df = plan_df.append([useful_df.iloc[0,:]],ignore_index=True)
# Drop used
    useful_df = useful_df.drop(useful_df.index[[0]])

    return plan_df, delta, proportions

In [33]:
proportions = np.array([prot_req/sum([prot_req,fat_req,carb_req]),fat_req/sum([prot_req,fat_req,carb_req]),carb_req/sum([prot_req,fat_req,carb_req])])
    # Sort full df by distance to initial proportions
useful_df['prop_distance'] = np.sqrt((useful_df.prot_prop-proportions[0])**2+(useful_df.fat_prop - proportions[1])**2+(useful_df.carb_prop - proportions[2])**2)
    #temp_df = temp_df.append(useful_df.iloc[randint(0,len(useful_df)-1),:])
useful_df = useful_df.sort_values(by=['prop_distance'])
plan_df = pd.DataFrame([useful_df.iloc[0,:]])
useful_df = useful_df.drop(useful_df.index[[0]])
delta = np.array([prot_req,fat_req,carb_req])-np.array([sum(plan_df['protein_g']),sum(plan_df['fat_g']),sum(plan_df['carb_g'])])
j = 0
while sum((abs(delta)<np.array([0.05*prot_req,0.05*fat_req,0.05*carb_req])))!=3:
    if sum(delta<0)==3:
        plan_df = plan_df.drop(plan_df.index[[randint(0,len(plan_df)-1)]])
    if j <= 4:
        plan_df, delta, proportions = add_sorted_item(useful_df[useful_df['rand_group']==j], plan_df, delta, proportions)
    else:
        plan_df, delta, proportions = add_sorted_item(useful_df, plan_df, delta, proportions)
    j+=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [34]:
plan_df

ndb_no                                          desc_only  kilocalories  \
0  45119257  HIGH LINER, SEA CUISINE, TORTILLA CRUSTED TILA...         280.0   
1  45232410  NATIVE AMERICAN NATURAL FOODS, BUFFALO MEAT TA...          70.0   
2  45206673  HORMEL, REV, #4 SMOKED TURKEY & PEPPERJACK CHE...         200.0   
3  45120813                BREADED CHICKEN BREAST TENDERLOINS,         170.0   
4  45205313  CLEAR SPRINGS FOODS, CRUNCHY POTATO CRUSTED MA...         280.0   
5  45199056                              CRISPY CHICKEN BITES,         180.0   
6  45314856                             CRISPY CHICKEN STRIPS,         230.0   
7  45120813                BREADED CHICKEN BREAST TENDERLOINS,         170.0   
8  45099287                            ORCHIDS, WHOLE OYSTERS,          80.0   
9  45061168                       HICKORY SMOKED STEAK STRIPS,         100.0   

   protein_g  fat_g  carb_g           unit_per_item  prot_prop  fat_prop  \
0         24     11      21  b'FILLET 148\xc2\xa0g'   0.428571  0.196429   
1          7      3       6      b'BAR 28\xc2\xa0g'   0.437500  0.187500   
2         16      8      15     b'WRAP 77\xc2\xa0g'   0.410256  0.205128   
3         15      6      14      b'ONZ 84\xc2\xa0g'   0.428571  0.171429   
4         25     10      22     b'ONZ 170\xc2\xa0g'   0.438596  0.175439   
5         15      7      15     b'ONZ 112\xc2\xa0g'   0.405405  0.189189   
6         18      9      17  b'STRIPS 112\xc2\xa0g'   0.409091  0.204545   
7         15      6      14      b'ONZ 84\xc2\xa0g'   0.428571  0.171429   
8          7      3       6      b'cup 56\xc2\xa0g'   0.437500  0.187500   
9          8      4       8      b'ONZ 28\xc2\xa0g'   0.400000  0.200000   

   carb_prop  prop_distance  rand_group  
0   0.375000       0.016362           3  
1   0.375000       0.021813           0  
2   0.384615       0.022927           1  
3   0.400000       0.014737           2  
4   0.385965       0.019733           3  
5   0.405405       0.015501           4  
6   0.386364       0.018542           4  
7   0.400000       0.010102           2  
8   0.375000       0.011136           3  
9   0.400000       0.000000           1

In [35]:
delta

array([2, 1, 2], dtype=int64)